In [1]:
import sqlalchemy
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# username = "gukh8079"
# passwd = "1a65fd1c0d48ac3524ac"
# host = "applied-sql.cs.colorado.edu"
# dbname = "gukh8079"

In [3]:
# db_string = "mysql://{0}:{1}@{2}/{3}".format(
#     username, passwd, host, dbname
# )
# print("Connection string is", db_string)

In [4]:
# try:
#     engine = sqlalchemy.create_engine( db_string );
#     conn = engine.connect()
# except Exception as exp:
#     print("Create engine failed:", exp)

In [5]:
# kills = conn.execute('''
#  Select concat('KILL ',id,';') from information_schema.processlist where user='gukh8079';
# ''').fetchall()
# kills

In [6]:
# print(str(kills[0]))
# for kill in kills:
#     command = str(kill).replace("(", "").replace(")", "").replace(",","").replace("'","")
#     print(command)
#     conn.execute(f"{command}").fetchall()

### Gurhar Khalsa Databases Final Project
* Video Link:
* Github Link:

* Github repo contains table initialization script, find functional dependencies and BCNF algorithms etc.





#### Motivation and learning outcomes:
    - To make a normalised database of olympic medalist data that supports the common queries one might perform on such a database
    - To gain a deeper understanding of the concepts learned in class regarding normalization, and good table schemas.
    - To practice the implementation of databases constraints, triggers, common queries etc.


* Because I wanted to design a normalized schema I made an BCNF normalizing algorithm that I applied to the flat csv data using pandas and python. I also made use of the closure computing algorithms provided to us in lecture in closure.py. The details are in findFD.ipynb, but here is some code examples:

* The algorithm below performs recursive a BCNF decomposition

```
def bcnf_decompose(R_attrs, verbose=False):
    # A relation R is in BCNF if and only if: 
    # whenever there is a nontrivial FD  A1 A2 ... An -> B1 B2 ... Bm for R,
    # it is the case that { A1, A2 , ... , An } is  a superkey for R

    attr_combos = list([set(x) for x in list(powerset(R_attrs)) if len(x) > 0])
    
    for attr_combo in attr_combos:
        
        x_closure = compute_closure(attr_combo, reversed(olympic_fds)).intersection(R_attrs)
        if not is_superkey(attr_combo, R_attrs, reversed(olympic_fds)) and x_closure != attr_combo:
            
            print(attr_combo, "bad fd", x_closure)
            
            if verbose:
                print("R_attrs:", R_attrs)
                print("X or attr_combo:", attr_combo)
         
                
            z = R_attrs - x_closure 
           
            y = x_closure - attr_combo
            
            if verbose:
                print("y :", y)
                print("z: ", z)
            
            r1 = bcnf_decompose(y.union(attr_combo))
            r2 = bcnf_decompose(z.union(attr_combo))
            
            return [r1, r2]
        
    return R_attrs
    
    
    
def bcnf(R_attrs, verbose=False):
    return bcnf_decompose(R_attrs, verbose)

```

* The set of functions below were used to generate the functional dependencies in the data:

```
def powerset(iterable):
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    

def check_fd(A_set, B, df):
    # If two tuples of R agree on all of the attributes A 1 ; A2 ;::: ;An 
    # (i.e., the tuples have the same values in their respective components for each of these attributes),  
    # then they must also agree on all of another list of attributes B 1 ; B2 ;::: ;Bm
    
    # For all values of A_set
    #    check if given a value of A_set, the B is same
    

    
    for group in df.groupby([*A_set])[B]:
        if len(group[1].unique()) > 1:

            return False
        
    return True
    
def get_all_candidates(df):
    candidate_list = []
    for tup in list(powerset(set(df.columns))):
        if set(tup) not in candidate_list and len(set(tup)) > 0:
            candidate_list.append(set(tup))
    return candidate_list
    
    
def get_all_fds(df):
    # Rational for just checking against singleton set
    # Bs: "It is common for the right side of an FD to be a single attribute. 
    # In fact,  we shall see that the one functional dependency A 1 A 2    A n -> B 1 B 2    B m 
    # is  equivalent to the set of FD's: 
    # A 1 A 2    A n -> B 1  
    # A 1 A 2    A n -> B 2  
    #    
    # A 1 A 2    A n -> B"
    # (Database Systems the Complete Book p. 68)
    fds = []
    a_candidates = get_all_candidates(df)
    b_candidates = set(df.columns)
    print("required iterations = {val}".format(val=len(a_candidates)*len(b_candidates)))
    i = 0
    for a_candidate in a_candidates:
        for b_candidate in b_candidates:
            i += 1
            if i % 100 == 0:
                print(i/len(a_candidates)*len(b_candidates), "% complete")
            if b_candidate not in a_candidate: # this check to make sure non-trivial
                if check_fd(a_candidate, b_candidate, df):
                    fds.append((a_candidate, b_candidate))
    return fds
    

```

* The normalized schema generated by the BCNF normalization algorithm was:

{'Discipline', 'Sport'},
{'City', 'Year'},
{'Athlete', 'Event', 'Gender'},
{'Athlete', 'Country', 'Event', 'Year'},
{'Athlete', 'Discipline', 'Event', 'Medal', 'Year'}


* The below E/R Diagram follows the schema output by my BCNF algorithm:

![Drag Racing](ERDiagram.png)

* Once I had the ER diagram of my normalized schema I got to work with writing table initialization code
* Note: I started with MySQL and was going to use MySQL workbench to forward engineer the database initialization of tables, but my colorado.edu mysql server kept crashing, so I switched to working with sqlite3 locally.

* The following demonstrates the initialization of database tables, population of the database, establishing triggers, insertion update and deletion functions etc.


### Lets take a look:

In [2]:
con = sqlite3.connect('test.db')
conn = con.cursor()

### Creating the tables

In [3]:
f = open("initializeTables.sql")
commandList = f.read().split(";")
commands = [(command.strip() +";").replace("\n","") for command in commandList]


In [4]:
for command in commands:
    conn.execute(command).fetchall()

##### Lets look at the tables

In [7]:
conn.execute("PRAGMA table_info(AthleteCountry);").fetchall()

[(0, 'Athlete', 'VARCHAR(45)', 1, None, 1),
 (1, 'Event', 'VARCHAR(45)', 1, None, 2),
 (2, 'Year', 'INT', 1, None, 3),
 (3, 'Country', 'VARCHAR(45)', 0, None, 0)]

In [8]:
conn.execute("PRAGMA table_info(AthleteEvent);").fetchall()

[(0, 'Athlete', 'VARCHAR(45)', 1, None, 1),
 (1, 'Event', 'VARCHAR(45)', 1, None, 2),
 (2, 'Gender', 'VARCHAR(45)', 0, None, 0)]

In [9]:
conn.execute("PRAGMA table_info(AthleteMedal);").fetchall()

[(0, 'Athlete', 'VARCHAR(45)', 1, None, 1),
 (1, 'Event', 'VARCHAR(45)', 1, None, 2),
 (2, 'Year', 'INT', 1, None, 3),
 (3, 'Discipline', 'VARCHAR(45)', 0, None, 5),
 (4, 'Medal', 'VARCHAR(45)', 0, None, 4)]

In [10]:
conn.execute("PRAGMA table_info(CityYear);").fetchall()

[(0, 'Year', 'INT', 1, None, 1), (1, 'City', 'VARCHAR(45)', 0, None, 0)]

In [11]:
conn.execute("PRAGMA table_info(Sports);").fetchall()

[(0, 'Sport', 'VARCHAR(45)', 1, None, 2),
 (1, 'Discipline', 'VARCHAR(45)', 1, None, 1)]


#### Now lets load the data in it's flat csv form

In [12]:
df = pd.read_csv("summer.csv")
df.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [13]:
len(df.Event.unique())

666

In [14]:
len(df.Discipline.unique())

67

In [15]:
len(df.Sport.unique())

43

In [16]:
df['Athlete'] = df['Athlete'].apply(lambda x: x.replace("'", "")) # Names like O'Connor break things

#### Functions to be used to populate the DB

In [17]:
def insert_AthleteCountry(obj):
    # ('Athlete',), ('Event',), ('Year',), ('Country',)
 
    conn.execute(f"""
        INSERT or IGNORE INTO AthleteCountry (Athlete, Event, Year, Country)
        VALUES('{obj['Athlete']}', '{obj['Event']}', '{obj['Year']}', '{obj['Country']}');
    """)
    


def insert_AthleteEvent(obj):
    # [('Athlete',), ('Event',), ('Gender',)]

    conn.execute(f"""
        INSERT or IGNORE INTO AthleteEvent (Athlete, Event, Gender)
        VALUES ('{obj['Athlete']}', '{obj['Event']}', '{obj['Gender']}');
    """)
    

def insert_AthleteMedal(obj):
    # [('Athlete',), ('Event',), ('Year',), ('Discipline',), ('Medal',)]

    conn.execute(f"""
        INSERT or IGNORE INTO AthleteMedal (Athlete, Event, Year, Discipline, Medal)
        VALUES ('{obj['Athlete']}', '{obj['Event']}', '{obj['Year']}', '{obj['Discipline']}', '{obj['Medal']}');
    """)
    

def insert_CityYear(obj):
  
    conn.execute(f"""
        INSERT or IGNORE INTO CityYear (Year, City)
        VALUES ('{obj['Year']}', '{obj['City']}');
    """)
    

def insert_Sports(obj):

        
    conn.execute(f"""
        INSERT or IGNORE INTO sports (Sport, Discipline)
        VALUES ('{obj['Sport']}', '{obj['Discipline']}');
    """)
    

    
    
    

def populate_database(df):
#     for index, obj in df.iterrows():
#         sports_has_AthleteMedal(obj)
        
    for index, obj in df.iterrows():
        insert_Sports(obj)
    for index, obj in df.iterrows():
        insert_AthleteEvent(obj)
    for index, obj in df.iterrows():
        insert_AthleteCountry(obj)
    for index, obj in df.iterrows():
        insert_AthleteMedal(obj)
    for index, obj in df.iterrows():
        insert_CityYear(obj)
    


### Populating DB

In [18]:
populate_database(df)

In [19]:
len(conn.execute("SELECT * FROM AthleteEvent;").fetchall())

26748

In [20]:
len(conn.execute("SELECT * FROM AthleteCountry;").fetchall())


31123

In [21]:
len(conn.execute("SELECT * FROM AthleteMedal;").fetchall())

31163

In [22]:
len(conn.execute("SELECT * FROM Sports;").fetchall())

67

In [23]:
conn.execute("SELECT * FROM CityYear;").fetchall()

[(1896, 'Athens'),
 (1900, 'Paris'),
 (1904, 'St Louis'),
 (1908, 'London'),
 (1912, 'Stockholm'),
 (1920, 'Antwerp'),
 (1924, 'Paris'),
 (1928, 'Amsterdam'),
 (1932, 'Los Angeles'),
 (1936, 'Berlin'),
 (1948, 'London'),
 (1952, 'Helsinki'),
 (1956, 'Melbourne / Stockholm'),
 (1960, 'Rome'),
 (1964, 'Tokyo'),
 (1968, 'Mexico'),
 (1972, 'Munich'),
 (1976, 'Montreal'),
 (1980, 'Moscow'),
 (1984, 'Los Angeles'),
 (1988, 'Seoul'),
 (1992, 'Barcelona'),
 (1996, 'Atlanta'),
 (2000, 'Sydney'),
 (2004, 'Athens'),
 (2008, 'Beijing'),
 (2012, 'London')]

### Some basic QA

In [24]:
def display_flat_form(conn, limit=None, verbose=False):
    if limit:
        res = conn.execute(f"""Select * From AthleteEvent Natural Join (SELECT * FROM AthleteMedal Natural Join
            (SELECT * FROM CityYear Natural Join 
                    (SELECT * FROM AthleteCountry NATURAL JOIN Sports
                        )
                        
                    ))
            LIMIT {limit};""").fetchall()
        if verbose:
            for row in res:
                print(row)
       
        return res
    else:
        res =  conn.execute("""Select * From AthleteEvent Natural Join (SELECT * FROM AthleteMedal Natural Join
            (SELECT * FROM AthleteCountry  Natural Join 
                    (SELECT * FROM CityYear NATURAL JOIN Sports
                        )
                        
                    ))
            ;""").fetchall()
        if verbose:
            for row in res:
                print(row)
                
        return res

def test_AthleteCountry(df, conn):
    
  
    db_res = conn.execute("""
        SELECT * FROM AthleteCountry;
    """).fetchall()
    
    if len(db_res) == len(df[['Athlete', 'Event', 'Year', 'Country']].drop_duplicates()):
        print("TEST 1 PASSED")
    else: 
        print("TEST 1 FAILED")

def test_join_size(df, conn):
   
    if len(df.drop_duplicates()) == len(display_flat_form(conn)):
        print("TEST 2 PASSED")
    else:
        print("TEST 2 FAILED")
        
def test_phelps(df, conn):
    df_res = len(df[(df['Athlete'] == "PHELPS, Michael") & (df['Medal'] == 'Gold')])
    
    db_res = len(conn.execute("""
        SELECT * FROM AthleteMedal Where Athlete = 'PHELPS, Michael' and Medal = 'Gold';
        """).fetchall())
    if df_res == db_res:
        print("TEST 3 PASSED (but orginal csv data is slightly off)")
    else:
        print("TEST 3 FAILED")
        
        
test_AthleteCountry(df, conn)
test_join_size(df, conn)
test_phelps(df, conn)


TEST 1 PASSED
TEST 2 PASSED
TEST 3 PASSED (but orginal csv data is slightly off)


In [25]:
def insert_new(conn, tup):
    # input should be an 9 tuple with form;
    # (Year, City, Sport, Discipline, Athlete, Country, Gender, Event, Medal) 
        if type(tup) == tuple:
            
            obj = {
                "Year": tup[0],
                "City": tup[1],
                "Sport": tup[2],
                "Discipline": tup[3],
                "Athlete": tup[4],
                "Country": tup[5],
                "Gender": tup[6],
                "Event": tup[7],
                "Medal": tup[8]
            }
        else:
            obj = tup # In the event that the input is already a dictionary
#         sports_has_AthleteMedal(obj)
        insert_Sports(obj)
        insert_AthleteEvent(obj)
        insert_AthleteCountry(obj)
        insert_AthleteMedal(obj)
        insert_CityYear(obj)

        
def delete(conn, tup):
    # Tup (Athlete, Event, Year, Medal)
    # Deletion needs to remove medalist and athlete from the dataset
    
    conn.execute(f"""
        DELETE FROM AthleteMedal
        WHERE Athlete = '{tup[0]}' AND Event = '{tup[1]}' AND Year = {tup[2]} AND Medal = '{tup[3]}';
        """)
   

def update_Medal(conn, Athlete, Year, Event, oldMedal, newMedal):
    # Retroactive changes are only to medals?
    if newMedal == None or newMedal == "None" or newMedal == "none":
        delete(conn, (Athlete, Event, Year, oldMedal))
    else:
        conn.execute(f"""
            UPDATE AthleteMedal
            SET Medal = '{newMedal}'
            WHERE Athlete = '{Athlete}' AND Year = {Year} AND Event = '{Event}' AND Medal = '{oldMedal}';
        """)
def name_change(conn, Athlete, newName):
    conn.execute(f"""
        Update AthleteCountry
        SET Athlete = '{newName}'
        WHERE Athlete = '{Athlete}';
    """)

#### Triggers

In [26]:
conn.execute("DROP TRIGGER IF EXISTS bef_athlete_medal_del;")
conn.execute("""
CREATE TRIGGER bef_athlete_medal_del BEFORE DELETE ON AthleteMedal
BEGIN

DELETE FROM AthleteCountry
        Where AthleteCountry.Athlete = OLD.Athlete AND AthleteCountry.Event = OLD.Event 
        AND AthleteCountry.Year = OLD.Year;

DELETE FROM AthleteEvent
        Where AthleteEvent.Athlete = OLD.Athlete AND AthleteEvent.Event = OLD.Event;


END;""")

conn.execute("DROP TRIGGER IF EXISTS athlete_name_change;")

conn.execute("""
CREATE TRIGGER athlete_name_change AFTER UPDATE OF Athlete ON AthleteCountry



BEGIN

Update AthleteMedal
SET Athlete = NEW.Athlete
WHERE Athlete = OLD.Athlete;


Update AthleteEvent
SET Athlete = NEW.Athlete
WHERE Athlete = OLD.Athlete;


END;""")

### Testing Insert, Delete and Update

#### Fictional Athlete wins olympic coding event 2 years in a row

In [27]:
insert_new(conn, {"Athlete":"Fictional Athlete",
                  "Event":"Coding",
                  "Year":2024,
                  "Discipline":"Computer Science",
                  "Sport": "Software Engineering",
                  "Country": "USA",
                  "Gender":"Male",
                  "City":"Los Angeles",
                  "Medal":"Gold"
                 })
insert_new(conn, {"Athlete":"Fictional Athlete",
                  "Event":"Coding",
                  "Year":2028,
                  "Discipline":"Computer Science",
                  "Sport": "Software Engineering",
                  "Country": "USA",
                  "Gender":"Male",
                  "City":"Los Angeles",
                  "Medal":"Gold"
                 })

In [28]:
conn.execute("Select * from AthleteMedal Where Athlete ='Fictional Athlete';").fetchall()

[('Fictional Athlete', 'Coding', 2024, 'Computer Science', 'Gold'),
 ('Fictional Athlete', 'Coding', 2028, 'Computer Science', 'Gold')]

In [29]:
conn.execute("Select * From AthleteCountry Where Athlete = 'Fictional Athlete';").fetchall()

[('Fictional Athlete', 'Coding', 2024, 'USA'),
 ('Fictional Athlete', 'Coding', 2028, 'USA')]

In [30]:
conn.execute("Select * From AthleteEvent Where Athlete = 'Fictional Athlete';").fetchall()

[('Fictional Athlete', 'Coding', 'Male')]

In [31]:
conn.execute("Select * from AthleteMedal Where Athlete ='PHELPS, Michael';").fetchall()

[('PHELPS, Michael', '100M Butterfly', 2004, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '100M Butterfly', 2008, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '100M Butterfly', 2012, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Butterfly', 2004, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Butterfly', 2008, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Butterfly', 2012, 'Swimming', 'Silver'),
 ('PHELPS, Michael', '200M Freestyle', 2004, 'Swimming', 'Bronze'),
 ('PHELPS, Michael', '200M Freestyle', 2008, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Individual Medley', 2004, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Individual Medley', 2008, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '200M Medley', 2012, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '400M Individual Medley', 2004, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '400M Individual Medley', 2008, 'Swimming', 'Gold'),
 ('PHELPS, Michael', '4X100M Freestyle', 2012, 'Swimming', 'Silver'),
 ('PHELPS, Michael', '4X100M Freestyle 

#### Above example suggests insertion function works ok

In [32]:
# Note the flexibility of the insert function to take a tuple or a dictionary as input
insert_new(conn, (2024, "Boulder", "Coding", "Computer Science", "Coder McCode", "USA", "Male", "Coding", "Bronze"))

In [33]:
conn.execute("Select * From AthleteEvent Where Athlete = 'Coder McCode';").fetchall()

[('Coder McCode', 'Coding', 'Male')]

In [34]:
name_change(conn, "Coder McCode", "Cody Coder")

In [35]:
conn.execute("Select * From Sports Where Sport = 'Coding';").fetchall()

[('Coding', 'Computer Science')]

In [36]:
conn.execute("Select * From AthleteCountry Where Athlete = 'Cody Coder';").fetchall()

[('Cody Coder', 'Coding', 2024, 'USA')]

In [37]:
conn.execute("Select * From AthleteMedal Where Athlete = 'Cody Coder';").fetchall()

[('Cody Coder', 'Coding', 2024, 'Computer Science', 'Bronze')]

In [38]:
conn.execute("Select * From AthleteMedal Where Discipline = 'Computer Science';").fetchall()

[('Fictional Athlete', 'Coding', 2024, 'Computer Science', 'Gold'),
 ('Fictional Athlete', 'Coding', 2028, 'Computer Science', 'Gold'),
 ('Cody Coder', 'Coding', 2024, 'Computer Science', 'Bronze')]

In [39]:
conn.execute("Select * From AthleteEvent Where Athlete = 'Cody Coder';").fetchall()

[('Cody Coder', 'Coding', 'Male')]

### The above sequence shows the cascade of a name change update to the other tables

In [40]:
delete(conn, ("Cody Coder", "Coding", 2024, "Bronze"))

In [41]:
conn.execute("Select * From AthleteMedal Where Event = 'Coding';").fetchall()

[('Fictional Athlete', 'Coding', 2024, 'Computer Science', 'Gold'),
 ('Fictional Athlete', 'Coding', 2028, 'Computer Science', 'Gold')]

In [42]:
conn.execute("Select * From Sports Where Discipline ='Computer Science';").fetchall()

[('Coding', 'Computer Science'), ('Software Engineering', 'Computer Science')]

In [43]:
conn.execute("Select * From AthleteMedal Where Athlete = 'Cody Coder';").fetchall()

[]

In [44]:
conn.execute("Select * From Sports Where Sport ='Coding';").fetchall()

[('Coding', 'Computer Science')]

In [45]:
conn.execute("Select * From AthleteCountry Where Athlete = 'Cody Coder';").fetchall()

[]

#### Above eg shows that the trigger correctly removes the athlete and cascades the result to other tables, but keeps the sport

In [46]:
insert_new(conn, (2028, "Boulder", "Coding", "Computer Science", 
                  "Coder McCode", "USA", "Male", "Coding", "Gold"))
insert_new(conn, (2028, "Boulder", "Coding", "Computer Science", 
                  "Other Coder", "USA", "Male", "Coding", "Silver"))

In [47]:
conn.execute("Select * From AthleteMedal Where Athlete='Coder McCode';").fetchall()

[('Coder McCode', 'Coding', 2028, 'Computer Science', 'Gold')]

* Lets say we strip Coder of his medal for using steroids

In [48]:

update_Medal(conn, 'Coder McCode', 2028, 'Coding', 'Gold', None)

In [49]:
conn.execute("Select * From AthleteMedal Where Athlete='Coder McCode';").fetchall()

[]

In [50]:
conn.execute("Select * From AthleteCountry Where Athlete='Coder McCode';").fetchall()

[]

In [51]:
conn.execute("Select * From AthleteEvent Where Athlete='Coder McCode';").fetchall()

[]

In [52]:
conn.execute("Select * From AthleteMedal Where Athlete = 'Other Coder';").fetchall()

[('Other Coder', 'Coding', 2028, 'Computer Science', 'Silver')]

#### now we promote Other Coder to Gold from Silver

In [53]:
# now we promote Other Coder to Gold from Silver
# conn, Athlete, Year, Event, oldMedal, newMedal
update_Medal(conn, "Other Coder", 2028, "Coding", "Silver", "Gold")

In [54]:
conn.execute("Select * From AthleteMedal Where Athlete = 'Other Coder';").fetchall()

[('Other Coder', 'Coding', 2028, 'Computer Science', 'Gold')]

### Creating Some Indexes

In [55]:
import time

conn.execute("""Drop Index if exists AthleteMedalIndex;""")
conn.execute("""Drop Index if exists AthleteEventIndex;""")

print("BEFORE INDEXES:")
start = time.perf_counter()
conn.execute("""
Select * From AthleteMedal group by Athlete, Event, Medal;
""")
end = time.perf_counter()
time_taken = end - start
print("Time taken:", time_taken)

start = time.perf_counter()
conn.execute("""
Select * From AthleteEvent group by Athlete, Event;
""")

end = time.perf_counter()
time_taken = end - start
print("Time taken:", time_taken)

BEFORE INDEXES:
Time taken: 0.01559748500585556
Time taken: 0.0004310780204832554


In [56]:
print("AFTER INDEXES")
conn.execute("""
Create index AthleteMedalIndex on AthleteMedal (Athlete, Event, Medal);
""").fetchall()
conn.execute("""
Create index AthleteEventIndex on AthleteEvent (Athlete, Event);
""").fetchall()


start = time.perf_counter()
conn.execute("""
Select * From AthleteMedal group by Athlete, Event, Medal;
""")
end = time.perf_counter()
time_taken = end - start
print("Time taken:", time_taken)

start = time.perf_counter()
conn.execute("""
Select * From AthleteEvent group by Athlete, Event;
""")

end = time.perf_counter()
time_taken = end - start
print("Time taken:", time_taken)

AFTER INDEXES
Time taken: 0.00016246899031102657
Time taken: 0.0005692869890481234


* Note the speedup provided by the indexes

### Grouping and Joining to answer queries about;
* Which countries have most medals?
* Which athletes have most medals?

In [57]:
# Using the below funcion to display the flat form like in the csv
display_flat_form(conn, limit=3)

[('HAJOS, Alfred',
  '100M Freestyle',
  'Men',
  1896,
  'Swimming',
  'Gold',
  'Athens',
  'HUN',
  'Aquatics'),
 ('HERSCHMANN, Otto',
  '100M Freestyle',
  'Men',
  1896,
  'Swimming',
  'Silver',
  'Athens',
  'AUT',
  'Aquatics'),
 ('DRIVAS, Dimitrios',
  '100M Freestyle For Sailors',
  'Men',
  1896,
  'Swimming',
  'Bronze',
  'Athens',
  'GRE',
  'Aquatics')]

In [58]:
# Country and medal Count pairs
conn.execute("""
    Select Country, Count(Medal) From (Select * From AthleteMedal Join AthleteCountry ON AthleteMedal.Athlete = AthleteCountry.Athlete 
    AND AthleteMedal.Event = AthleteCountry.Event) Group By Country
    Order by Count(Medal) Desc Limit 10;
""").fetchall()

[('USA', 5970),
 ('URS', 2971),
 ('GBR', 2049),
 ('ITA', 1996),
 ('HUN', 1795),
 ('GER', 1794),
 ('FRA', 1774),
 ('AUS', 1700),
 ('SWE', 1364),
 ('NED', 1178)]

In [59]:
# Same thing but for gold medals
conn.execute("""
    Select Country, Count(Medal) From (Select * From AthleteMedal Join AthleteCountry ON AthleteMedal.Athlete = AthleteCountry.Athlete 
    AND AthleteMedal.Event = AthleteCountry.Event
    Where Medal = 'Gold') Group By Country
    Order by Count(Medal) Desc Limit 10;
""").fetchall()

[('USA', 3072),
 ('URS', 1289),
 ('HUN', 795),
 ('ITA', 754),
 ('GER', 675),
 ('GBR', 674),
 ('FRA', 566),
 ('AUS', 446),
 ('SWE', 445),
 ('GDR', 439)]

In [60]:
# Ten most successful olympians
conn.execute("""
Select Athlete, Count(Athlete), Discipline From (Select * From AthleteMedal as am Join Sports On am.Discipline = Sports.Discipline) Group by Athlete
Order By Count(Athlete) Desc Limit 10;
""").fetchall()

[('PHELPS, Michael', 22, 'Swimming'),
 ('LATYNINA, Larisa', 18, 'Artistic G.'),
 ('ANDRIANOV, Nikolay', 15, 'Artistic G.'),
 ('MANGIAROTTI, Edoardo', 13, 'Fencing'),
 ('ONO, Takashi', 13, 'Artistic G.'),
 ('SHAKHLIN, Boris', 13, 'Artistic G.'),
 ('COUGHLIN, Natalie', 12, 'Swimming'),
 ('FISCHER, Birgit', 12, 'Canoe / Kayak F'),
 ('KATO, Sawao', 12, 'Artistic G.'),
 ('NEMOV, Alexei', 12, 'Artistic G.')]

### Conlcuding discussion
* We saw an algorithm that results in creating a normalized schema for the database.
* We have seen from the series of queries and function calls above that insertion, deletion and updates properly result in cascades and trigger calls. 
* We have also seen how the database provides an interface for performing queries that join and group the data to answer questions pertaining to sports statistics, which was one of the main goals.